# importing packages

In [36]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm

np.random.seed(2022)

# relhum conversion functions

In [28]:
def eliq(T):
    a_liq = np.float32(np.array([-0.976195544e-15,-0.952447341e-13,\
                                 0.640689451e-10,\
                      0.206739458e-7,0.302950461e-5,0.264847430e-3,\
                      0.142986287e-1,0.443987641,6.11239921]));
    c_liq = np.float32(-80.0)
    T0 = np.float32(273.16)
    return np.float32(100.0)*np.polyval(a_liq,np.maximum(c_liq,T-T0))

def eice(T):
    a_ice = np.float32(np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                      0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                      0.188439774e-1,0.503160820,6.11147274]));
    c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
    T0 = np.float32(273.16)
    return np.where(T>c_ice[0],eliq(T),\
                   np.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),\
                           np.float32(100.0)*np.polyval(a_ice,T-T0)))

def esat(T):
    T0 = np.float32(273.16)
    T00 = np.float32(253.16)
    omtmp = (T-T00)/(T0-T00)
    omega = np.maximum(np.float32(0.0),np.minimum(np.float32(1.0),omtmp))
    return np.where(T>T0,eliq(T),np.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))

def RH(T,qv,P0,PS,hyam,hybm):
    R = np.float32(287.0)
    Rv = np.float32(461.0)
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    T = np.float32(T)
    qv = np.float32(qv)
    p = np.float32(p)
    
    return Rv*p*qv/(R*esat(T))

In [35]:
datasets = !ls
datasets = [x for x in datasets if "h1" in x]
spData = xr.open_mfdataset(datasets)

nntbp = spData["NNTBP"].values
nnqbp = spData["NNQBP"].values
p0 = spData["P0"].values
ps = spData["NNPS"].values
hyam = spData["hyam"].values
hybm = spData["hybm"].values
relhum = spData["RELHUM"].values
tphystnd = spData["TPHYSTND"].values
phq = spData["PHQ"].values

p0 = np.array(list(set(p0)))

newhum = np.zeros((spData["time"].shape[0],\
                   spData["lev"].shape[0], \
                   spData["lat"].shape[0], \
                   spData["lon"].shape[0]))

lats = spData["lat"]
lons = spData["lon"]

In [34]:
for i in tqdm(range(len(lats))):
    for j in range(len(lons)):
        latIndex = i
        lonIndex = j
        R = np.float32(287.0)
        Rv = np.float32(461.0)
        p = np.float32(p0 * hyam + ps[:, None, latIndex, lonIndex] * hybm) # Total pressure (Pa)

        T = np.float32(nntbp[:, :, latIndex, lonIndex])
        qv = np.float32(nnqbp[:, :, latIndex, lonIndex])
        p = np.float32(p)
        newhum[:,:, latIndex, lonIndex] = np.float32(Rv*p*qv/(R*esat(T)))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:04<00:00, 15.91it/s]


In [18]:
yas

array([[1.9190507e-05, 1.2547281e-04, 8.9550420e-04, ..., 9.6153301e-01,
        9.4901347e-01, 9.6164906e-01],
       [1.9101750e-05, 1.2458619e-04, 8.9724018e-04, ..., 9.6226490e-01,
        9.4642472e-01, 9.5881152e-01],
       [1.8993975e-05, 1.2352996e-04, 8.9667720e-04, ..., 9.6303856e-01,
        9.4527107e-01, 9.5512652e-01],
       ...,
       [2.1175736e-05, 1.3348136e-04, 1.2820862e-03, ..., 8.8512754e-01,
        9.3123478e-01, 9.6986449e-01],
       [2.1163949e-05, 1.3336072e-04, 1.2869698e-03, ..., 8.8269174e-01,
        9.3080842e-01, 9.7888279e-01],
       [2.1079930e-05, 1.3318541e-04, 1.2913774e-03, ..., 8.7168556e-01,
        9.3530428e-01, 9.7838253e-01]], dtype=float32)

In [12]:
#esat



In [ ]:
#eliq
eliq(T)


In [ ]:
#eice



# data processing functions

In [ ]:
def doMonth(month):
    datasets = !ls
    n = str(month)
    datasets = [x for x in datasets if "h1.0000-" + n.zfill(2) in x]
    return xr.open_mfdataset(datasets)

def makeSuffix(month):
    n = str(month)
    return "_" + n.zfill(2)

def saveNNInput(month):
    spData = doMonth(month)
    suffix = makeSuffix(month)
    print("read in data")
    nntbp = spData["NNTBP"].values
    nnqbp = spData["NNQBP"].values
    p0 = spData["P0"].values
    ps = spData["NNPS"].values
    hyam = spData["hyam"].values
    hybm = spData["hybm"].values
    relhum = spData["RELHUM"].values
    tphystnd = spData["TPHYSTND"].values
    phq = spData["PHQ"].values

    p0 = np.array(list(set(p0)))
    print("loaded in data")
    newhum = np.zeros((spData["time"].shape[0],\
                         spData["lev"].shape[0], \
                         spData["lat"].shape[0], \
                         spData["lon"].shape[0]))
    lats = spData["lat"]
    lons = spData["lon"]
    print("starting for loop")
    for i in tqdm(range(len(lats))):
        for j in range(len(lons)):
            latIndex = i
            lonIndex = j
            R = np.float32(287.0)
            Rv = np.float32(461.0)
            p = np.float32(p0 * hyam + ps[:, None, latIndex, lonIndex] * hybm) # Total pressure (Pa)

            T = np.float32(nntbp[:, :, latIndex, lonIndex])
            qv = np.float32(nnqbp[:, :, latIndex, lonIndex])
            p = np.float32(p)
            newhum[:,:, latIndex, lonIndex] = np.float32(Rv*p*qv/(R*esat(T)))
    
    nntbp = np.moveaxis(nntbp[1:,:,:,:],0,1)
    print("nntbp")
    print(nntbp.shape)
    
    nnqbp = np.moveaxis(nnqbp[1:,:,:,:],0,1)
    print("nnqbp")
    print(nnqbp.shape)
    
    lhflx = spData["LHFLX"].values[np.newaxis,:-1,:,:]
    print("lhflx")
    print(lhflx.shape)
    
    shflx = spData["SHFLX"].values[np.newaxis,:-1,:,:]
    print("shflx")
    print(shflx.shape)
    
    ps = spData["NNPS"].values[np.newaxis,1:,:,:]
    print("ps")
    print(ps.shape)
    
    solin = spData["SOLIN"].values[np.newaxis,1:,:,:]
    print("solin")
    print(solin.shape)
    
    newhum = np.moveaxis(newhum[1:,:,:,:],0,1)
    print("newhum")
    print(newhum.shape)
    
    oldhum = np.moveaxis(relhum[1:,:,:,:],0,1)
    print("oldhum")
    print(oldhum.shape)
    
    tphystnd = np.moveaxis(tphystnd[1:,:,:,:],0,1)
    print("tphystnd")
    print(tphystnd.shape)
    

    phq = np.moveaxis(phq[1:,:,:,:],0,1)
    print("phq")
    print(phq.shape)

    nnInput = np.concatenate((nntbp, \
                              nnqbp, \
                              lhflx, \
                              shflx, \
                              ps, \
                              solin, \
                              newhum, \
                              oldhum, \
                              tphystnd, \
                              phq))
    print("nnInput")
    nnInput.shape

    errors = (newhum-oldhum/100).flatten()
    result = "Mean error: " + str(np.mean(errors)) + "\n"
    result = result + "Variance: " + str(np.var(errors)) + "\n"
    result = result + "nntbp.shape: " + str(nntbp.shape) + "\n"
    result = result + "nnqbp.shape: " + str(nnqbp.shape) + "\n"
    result = result + "lhflx.shape: " + str(lhflx.shape) + "\n"
    result = result + "shflx.shape: " + str(shflx.shape) + "\n"
    result = result + "ps.shape: " + str(ps.shape) + "\n"
    result = result + "solin.shape: " + str(solin.shape) + "\n"
    result = result + "newhum.shape: " + str(newhum.shape) + "\n"
    result = result + "oldhum.shape: " + str(oldhum.shape) + "\n"
    result = result + "tphystnd.shape: " + str(tphystnd.shape) + "\n"
    result = result + "phq.shape: " + str(phq.shape) + "\n"
    result = result + "nnInput.shape: " + str(nnInput.shape) + "\n"
    print(result)
    
    #added 32 bit fix
    nnInput = np.float32(nnInput)
    
    fileName = 'nnInput' + suffix + '.npy'
    with open(fileName, 'wb') as f:
        np.save(f, nnInput)

    diagnostics = 'diagnostics' + suffix + '.txt'
    with open(diagnostics, 'a') as fp:
        fp.write(result)

# run the code

In [ ]:
#saveNNInput(1)

In [ ]:
#saveNNInput(2)

In [ ]:
#saveNNInput(3)

In [ ]:
#saveNNInput(4)

In [ ]:
#saveNNInput(5)

In [ ]:
#saveNNInput(6)

In [ ]:
#saveNNInput(7)

In [ ]:
#saveNNInput(8)

In [ ]:
#saveNNInput(9)

In [ ]:
#saveNNInput(10)

In [ ]:
saveNNInput(11)

In [ ]:
saveNNInput(12)